In [56]:
import re
import json
import random

In [229]:
template_fib = """
{
    "模板": "填空题",
    "题干": "计算<x*a>÷<b>×<z*c>÷<a>×<y*b>÷<c>=___。",
    "知识点": "乘法交换律",
    "思路": "可以将原式化简，化成分数的形式，然后再约分，不难求得结果。",
    "答案": "<x*y*z>",
    "解析": "解：根据分析，原式=<x*a>÷<b>×<z*c>÷<a>×<y*b>÷<c>\
            =(<x*a>×<y*b>×<z*b>)÷(<a>×<b>×<c>)\
            =<x*y*z>",
    "变量": {
        "x": "random int from 1 to 3",
        "y": "random int from 2 to 4",
        "z": "random int from 1 to 2",
        "a": "random int from 100 to 333",
        "b": "random int from 50 to 250",
        "c": "random int from 100 to 433"
    }
}
"""

In [230]:
fib_raw = json.loads(template_fib)

In [231]:
print(fib_raw)

{'模板': '填空题', '题干': '计算<x*a>÷<b>×<z*c>÷<a>×<y*b>÷<c>=___。', '知识点': '乘法交换律', '思路': '可以将原式化简，化成分数的形式，然后再约分，不难求得结果。', '答案': '<x*y*z>', '解析': '解：根据分析，原式=<x*a>÷<b>×<z*c>÷<a>×<y*b>÷<c>            =(<x*a>×<y*b>×<z*b>)÷(<a>×<b>×<c>)            =<x*y*z>', '变量': {'x': 'random int from 1 to 3', 'y': 'random int from 2 to 4', 'z': 'random int from 1 to 2', 'a': 'random int from 100 to 333', 'b': 'random int from 50 to 250', 'c': 'random int from 100 to 433'}}


In [260]:
class QuestionTemplate:
    def __init__(self, params):
        self.template = params['模板']
        self.__body_raw = params['题干']
        self.knowledge = params['知识点']
        self.__hint_raw = params['思路']
        self.__key_raw = params['答案']
        self.__explanation_raw = params['解析']
        self.__variables_raw = params['变量']
        
        self.variables = self.__parse_variable()
        self.body = self.__content_constructor(self.__body_raw)
        self.hint = self.__content_constructor(self.__hint_raw)
        self.key = self.__content_constructor(self.__key_raw)
        self.explanation = self.__content_constructor(self.__explanation_raw)
        
    def roll(self):
        var = self.__parse_variable()
        self.variables = var
        
    def generate(self):
        pass
            
    def __content_constructor(self, content):
        expressions = self.__expression_parser(content)
        content_wo_data = self.__string_constructor(content)
        calced_expressions = self.__expr_calc(expressions)
        return content_wo_data.format(*calced_expressions)
    
    def __expr_calc(self, exprs):
        var_name = list(self.__variables_raw.keys())
        for v in var_name:
            exprs = [re.sub(v, str(self.variables[v]), e) for e in exprs]
        result = [eval(e) for e in exprs]
        return result
    
    def __expression_parser(self, content):
        pattern = r'(?<=<).+?(?=>)+?'
        return re.findall(pattern, content)
    
    def __string_constructor(self, content):
        pattern = r'(<.+?>)+?'
        replacement = '{}'
        return re.sub(pattern, replacement, content)
    
    def __parse_variable(self):
        variables = self.__variables_raw.copy()
        for key, value in variables.items():
            variables[key] = self.__rule_parser(value)
        return variables

    def __rule_parser(self, rule):
        tokens = rule.split()
        d_type = 'int' # use int if not specified
        if 'float' in tokens:
            d_type = 'float'
            
        use_random = 'random' in tokens # use random number if random is found in tokens
        if use_random:
            assert('from' in tokens and 'to' in tokens)
            floor = int(re.findall(r'(?<=from).(\d+)', rule)[0])
            ceil = int(re.findall(r'(?<=to).(\d+)',rule)[0])
            if d_type == 'int':
                return random.randint(floor, ceil)
            else:
                return random.random * (ceil - floor) + floor
        else:
            number = re.findall(r'\d+', rule)[0]
            return eval('{}(number)'.format(d_type))

In [261]:
template = QuestionTemplate(fib_raw)

['x*a', 'b', 'z*c', 'a', 'y*b', 'c']
计算{}÷{}×{}÷{}×{}÷{}=___。
[222, 230, 125, 111, 920, 125]
[]
可以将原式化简，化成分数的形式，然后再约分，不难求得结果。
[]
['x*y*z']
{}
[8]
['x*a', 'b', 'z*c', 'a', 'y*b', 'c', 'x*a', 'y*b', 'z*b', 'a', 'b', 'c', 'x*y*z']
解：根据分析，原式={}÷{}×{}÷{}×{}÷{}            =({}×{}×{})÷({}×{}×{})            ={}
[222, 230, 125, 111, 920, 125, 222, 920, 230, 111, 230, 125, 8]


In [241]:
template.variables["a"]

295

In [224]:
template.roll()